In [9]:
import mayavi.mlab as mlab
%gui qt
import argparse
import copy
import json
import os
import sys
import pandas as pd
try:
    import apex
except:
    print("No APEX!")
import numpy as np
import torch
import yaml
from det3d import torchie
from det3d.datasets import build_dataloader, build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    batch_processor,
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
)
from det3d.torchie.trainer import load_checkpoint
import pickle 
import time 
from matplotlib import pyplot as plt 
from det3d.torchie.parallel import collate, collate_kitti
from torch.utils.data import DataLoader
from pyquaternion import Quaternion
import matplotlib.cm as cm
import subprocess
import cv2
from tools.demo_utils import visual 
from collections import defaultdict
import logging
import itertools
import pickle as pkl
def load_pkl_dictionary(pkl_fpath):
    with open(pkl_fpath, "rb") as f:
        return pkl.load(f)

In [18]:
from types import SimpleNamespace
from argoverse.utils.json_utils import read_json_file, save_json_dict
from argoverse.utils.pkl_utils import load_pkl_dictionary, save_pkl_dictionary
import torch.distributed as dist

from centerpoint.utils.config import Config
from centerpoint.registry import DETECTORS

from centerpoint.dataset.centerpoint_dataloader import build_dataloader
from centerpoint.nuscenes_dataset import NuScenesDataset
from centerpoint.nuscenes_dataset import Reformat
from centerpoint.pcd import PCDDataset
from centerpoint.utils.checkpoint import load_checkpoint
from centerpoint.utils.compose import Compose
from centerpoint.utils.loading import LoadPointCloudAnnotations, LoadPointCloudFromFile
from centerpoint.utils.preprocess import AssignLabel, Preprocess
from centerpoint.utils.test_aug import DoubleFlip
from centerpoint.utils.voxel_generator import Voxelization

In [28]:
def get_dist_info():
    if torch.__version__ < "1.0":
        initialized = dist._initialized
    else:
        initialized = dist.is_initialized()
    if initialized:
        rank = dist.get_rank()
        world_size = dist.get_world_size()
    else:
        rank = 0
        world_size = 1
    return rank, world_size


def get_root_logger(log_level=logging.INFO):
    logger = logging.getLogger()
    if not logger.hasHandlers():
        logging.basicConfig(
            format="%(asctime)s - %(levelname)s - %(message)s", level=log_level
        )
    rank, _ = get_dist_info()
    if rank != 0:
        logger.setLevel("ERROR")
    return logger


def example_to_device(example, device=None, non_blocking=False) -> dict:
    """ """
    assert device is not None

    example_torch = {}
    float_names = ["voxels", "bev_map"]
    for k, v in example.items():
        if k in ["anchors", "anchors_mask", "reg_targets", "reg_weights", "labels"]:
            example_torch[k] = [res.to(device, non_blocking=non_blocking) for res in v]
        elif k in [
            "voxels",
            "bev_map",
            "coordinates",
            "num_points",
            "points",
            "num_voxels",
        ]:
            example_torch[k] = v.to(device, non_blocking=non_blocking)
        elif k == "calib":
            calib = {}
            for k1, v1 in v.items():
                calib[k1] = torch.tensor(v1).to(device, non_blocking=non_blocking)
            example_torch[k] = calib
        else:
            example_torch[k] = v

    return example_torch


def parse_second_losses(losses):
    """ """
    log_vars = OrderedDict()
    loss = sum(losses["loss"])
    for loss_name, loss_value in losses.items():
        if loss_name == "loc_loss_elem":
            log_vars[loss_name] = [[i.item() for i in j] for j in loss_value]
        else:
            log_vars[loss_name] = [i.item() for i in loss_value]

    return loss, log_vars


def batch_processor(model, data, train_mode, **kwargs):
    """ """
    if "local_rank" in kwargs:
        device = torch.device(kwargs["local_rank"])
    else:
        device = None

    example = example_to_device(data, device, non_blocking=False)

    del data

    if train_mode:
        losses = model(example, return_loss=True)
        loss, log_vars = parse_second_losses(losses)

        outputs = dict(
            loss=loss, log_vars=log_vars, num_samples=len(example["anchors"][0])
        )
        return outputs
    else:
        return model(example, return_loss=False)




def build_detector(logger, cfg, train_cfg=None, test_cfg=None):

    registry = DETECTORS
    default_args=dict(train_cfg=train_cfg, test_cfg=test_cfg)
    return build_from_cfg(logger, cfg, registry, default_args)


def is_str(x):
    """Whether the input is an string instance."""
    return isinstance(x, six.string_types)

def build_from_cfg(logger, cfg, registry, default_args=None):
    """Build a module from config dict.
    Args:
        cfg (dict): Config dict. It should at least contain the key "type".
        registry (:obj:`Registry`): The registry to search the type from.
        default_args (dict, optional): Default initialization arguments.
    Returns:
        obj: The constructed object.
    """

    from center_head import CenterHead
    from centerpoint.models.scn_backbone import SpMiddleResNetFHD
    from centerpoint.models.rpn import RPN
    from centerpoint.models.voxel_encoder import VoxelFeatureExtractorV3
    from centerpoint.models.voxelnet import VoxelNet

    reader = VoxelFeatureExtractorV3(
        num_input_features = 5,
        norm_cfg = None
    )
    backbone = SpMiddleResNetFHD(
        num_input_features = 5,
        ds_factor = 8,
        norm_cfg = None
    )
    neck = RPN(
        layer_nums = [5, 5],
        ds_layer_strides = [1, 2],
        ds_num_filters = [128, 256],
        us_layer_strides = [1, 2],
        us_num_filters = [256, 256],
        num_input_features = 256,
        norm_cfg = None,
        logger = logger # <Logger RPN (INFO)>
    )
    bbox_head = CenterHead(
        in_channels = sum([256, 256]),
        tasks=[
            dict(num_class=3, class_names=['VEHICLE', 'PEDESTRIAN', 'CYCLIST']),
        ],
        dataset = 'waymo',
        weight = 0.25,
        code_weights = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 0.2, 1.0, 1.0],
        common_heads = {
            'reg': (2, 2),
            'height': (1, 2),
            'dim': (3, 2),
            'rot': (2, 2),
            'vel': (2, 2)
        },
        share_conv_channel = 64,
        dcn_head = False,
    )

    detector = VoxelNet(
        reader=reader,
        backbone=backbone,
        neck=neck,
        bbox_head=bbox_head,
        train_cfg=None,
        test_cfg=SimpleNamespace(**default_args['test_cfg']),
        pretrained=None
    )
    return detector


def build_dataset(cfg, args):
    """ """
    nsweeps = cfg.nsweeps
    dataset_name = cfg.type
    split = args['split']
    if split == 'test':
        info_path = f'data/{dataset_name}/infos_test_{str(nsweeps).zfill(2)}sweeps_withvelo.pkl'
    else:
        info_path = f'data/{dataset_name}/infos_val_{str(nsweeps).zfill(2)}sweeps_withvelo_filter_True.pkl'
    info_path = cfg.root_path

    pipeline = [
            LoadPointCloudFromFile(dataset = 'PCDDataset')
    ]
    if split != 'test':
        # only relevant for train or val
        pipeline += [LoadPointCloudAnnotations(with_bbox = True)]

    pipeline.extend([
            Preprocess(
                cfg=SimpleNamespace(**{
                    'mode': 'val',
                    'shuffle_points': False,
                    'remove_environment': False,
                    'remove_unknown_examples': False
                })
            ),
            Voxelization(
                cfg = cfg['pipeline'][3]['cfg']
            ),
            AssignLabel(
                cfg = cfg['pipeline'][4]['cfg']
            ),
            Reformat(double_flip=False) #Changed from True
        ])
    dataset = PCDDataset(
        info_path = info_path,
        root_path = "data/Panasonic/processed/lidar",
        test_mode = True,
        class_names = cfg.class_names,
        nsweeps = nsweeps,
        ann_file = info_path,
        pipeline = pipeline
    )
    return dataset

def load_opts():
    """ """
    opts_dict = {
        'config': 'configs/mixed_pcd.py',#nusc_centerpoint_voxelnet_dcn_0075voxel_flip_testset.py',
        'work_dir': 'work_dirs/mixed_wn',
        'checkpoint': 'work_dirs/mixed_wn/latest.pth',
        'txt_result': False,
        'gpus': 1,
        'launcher': 'none',
        'speed_test': True,
        'local_rank': 0,
        'testset': False
    }
    opts = SimpleNamespace(**opts_dict)

    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(opts.local_rank)

    return opts

In [26]:
args = {'split':'test', 'dataset_name':'pcd', 'pkl_save_fname':'2020-01-13-argoverse_test_prediction.pkl', 'nsweeps':2}

In [29]:
opts = load_opts()

# pdb.set_trace()
cfg = Config.fromfile(opts.config)
cfg.local_rank = opts.local_rank

# update configs according to CLI args
if opts.work_dir is not None:
    cfg.work_dir = opts.work_dir

distributed = False
if "WORLD_SIZE" in os.environ:
    distributed = int(os.environ["WORLD_SIZE"]) > 1

cfg.gpus = opts.gpus

# init logger before other steps
logger = get_root_logger(cfg.log_level)
logger.info("Distributed testing: {}".format(distributed))
logger.info(f"torch.backends.cudnn.benchmark: {torch.backends.cudnn.benchmark}")

# pdb.set_trace()
print('Model = ', cfg.model)
model = build_detector(logger, cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)

if opts.testset:
    print("Use Test Set")
    dataset = build_dataset(cfg.data.test, args)
else:
    print("Use Val Set")
    dataset = build_dataset(cfg.data.val, args)

data_loader = build_dataloader(
    dataset,
    batch_size=cfg.data.samples_per_gpu if not opts.speed_test else 1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=distributed,
    shuffle=False,
)

# pdb.set_trace()
checkpoint = load_checkpoint(model, opts.checkpoint, map_location="cpu")

model = model.cuda()
model.eval()
mode = "val"

logger.info(f"work dir: {opts.work_dir}")
# if cfg.local_rank == 0:
#     prog_bar = torchie.ProgressBar(len(data_loader.dataset) // cfg.gpus)

detections = {}
cpu_device = torch.device("cpu")

2021-02-16 19:29:38,968 - INFO - Distributed testing: False
2021-02-16 19:29:38,969 - INFO - torch.backends.cudnn.benchmark: False
2021-02-16 19:29:39,022 - INFO - Finish RPN Initialization
2021-02-16 19:29:39,022 - INFO - num_classes: [3]
2021-02-16 19:29:39,031 - INFO - Finish CenterHead Initialization


Model =  {'type': 'VoxelNet', 'pretrained': None, 'reader': {'type': 'VoxelFeatureExtractorV3', 'num_input_features': 5}, 'backbone': {'type': 'SpMiddleResNetFHD', 'num_input_features': 5, 'ds_factor': 8}, 'neck': {'type': 'RPN', 'layer_nums': [5, 5], 'ds_layer_strides': [1, 2], 'ds_num_filters': [128, 256], 'us_layer_strides': [1, 2], 'us_num_filters': [256, 256], 'num_input_features': 256, 'logger': <Logger RPN (INFO)>}, 'bbox_head': {'type': 'CenterHead', 'in_channels': 512, 'tasks': [{'num_class': 3, 'class_names': ['VEHICLE', 'PEDESTRIAN', 'CYCLIST']}], 'dataset': 'waymo', 'weight': 2, 'code_weights': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 0.2, 1.0, 1.0], 'common_heads': {'reg': (2, 2), 'height': (1, 2), 'dim': (3, 2), 'rot': (2, 2), 'vel': (2, 2)}}}
Use HM Bias:  -2.19
Use Val Set
Preprocess cfg =  namespace(mode='val', remove_environment=False, remove_unknown_examples=False, shuffle_points=False)

                Voxel cfg: voxel_size - [0.1, 0.1, 0.15]; point_cloud_range - [-75.2,

2021-02-16 19:29:40,610 - INFO - work dir: work_dirs/mixed_wn


In [48]:
start = time.time()

start = int(len(dataset) / 3)
end = int(len(dataset) * 2 /3)

time_start = 0 
time_end = 0 
points_list = [] 

for i, data_batch in enumerate(data_loader):
    """
    data_batch will have collated examples
    so for double flip, get an extra dimension on the outside, indexed with 0, 1, 2, 3 etc for 4 examples
    """
    print(f'{i}/{len(data_loader)}')
    if i == start:
        torch.cuda.synchronize()
        time_start = time.time()

    if i == end:
        torch.cuda.synchronize()
        time_end = time.time()

    with torch.no_grad():
        outputs = batch_processor(
            model, data_batch, train_mode=False, local_rank=opts.local_rank,
        )
    points = data_batch['points'][:, 1:4].cpu().numpy()
    points_list.append(points.T)
    for output in outputs:

        # `output` is a dictionary with keys
        # dict_keys(['box3d_lidar', 'scores', 'label_preds', 'metadata'])
        # box3d_lidar is a tensor of shape [217, 9]
        # scores is a tensor of shape [217]
        # label_preds is a tensor of shape [217]
        # metadata has {
        #     'image_prefix': PosixPath('data/nuScenes/v1.0-test'),
        #     'num_point_features': 5,
        #     'token': '3c61eda50f694585bac0614e4660eca1'
        #    }
        token = output["metadata"]["token"]
        detections[token] = data_batch["annos"]
        for k, v in output.items():
            if k not in ["metadata"]:
                output[k] = v.to(cpu_device)
        detections.update(
            {token: output,}
        )
        detections[token]["annos"] = data_batch["annos"]
        # optionally dump the input!
        # detections[token]["input_coordinates"] = data_batch["coordinates"].to(cpu_device)
        # detections[token]["input_voxels"] = data_batch["voxels"].to(cpu_device)
all_predictions = detections
all_predictions = [all_predictions[i] for i in all_predictions]
print("\n Total time per frame: ", (time_end -  time_start) / (end - start))

0/2
1/2

 Total time per frame:  0.13686704635620117


In [49]:
RED = (1,0,0)
GREEN = (0,0.5,0)
s = pd.Series(data=['VEHICLE', 'PEDESTRIAN', 'CYCLIST'], index=[0, 1, 2])
for i in range(len(points_list)):
    fig = draw_lidar(np.array(points_list[i]).T)
    fig = draw_gt_boxes3d(convert_detbox_to_coordinates([all_predictions[i]])[0], fig, color=GREEN, labels= s[np.array(all_predictions[i]['label_preds'])].values, scores = all_predictions[i]['scores'].tolist())

PEDESTRIAN,VEHICLE,VEHICLE,VEHICLE,VEHICLE,VEHICLE,PEDESTRIAN,VEHICLE,PEDESTRIAN,PEDESTRIAN,PEDESTRIAN,VEHICLE,VEHICLE,VEHICLE,VEHICLE,VEHICLE,VEHICLE,VEHICLE,PEDESTRIAN,VEHICLE,PEDESTRIAN,VEHICLE,PEDESTRIAN,VEHICLE,PEDESTRIAN,VEHICLE,PEDESTRIAN,VEHICLE,VEHICLE,VEHICLE,VEHICLE,VEHICLE,VEHICLE,PEDESTRIAN,VEHICLE,PEDESTRIAN,PEDESTRIAN,PEDESTRIAN,VEHICLE,VEHICLE,PEDESTRIAN,VEHICLE,PEDESTRIAN,CYCLIST,PEDESTRIAN,VEHICLE,PEDESTRIAN,PEDESTRIAN,

In [47]:
[all_predictions[i] for i in all_predictions]

[{'box3d_lidar': tensor([[ 1.2645e+00, -1.1032e+01,  1.3959e+00,  7.6347e-01,  7.7269e-01,
            1.7250e+00,  1.6645e-02, -8.2055e-01, -9.7078e-02],
          [ 2.9259e+00,  1.0629e+01,  6.9083e-01,  1.9295e+00,  4.4831e+00,
            1.4498e+00,  5.6300e-03, -1.5117e-04, -2.9115e-02],
          [ 1.6247e+01,  6.9149e+00,  1.1888e+00,  2.5003e+00,  6.1090e+00,
            2.7842e+00,  1.3063e-03, -1.8473e-04,  1.5698e+00],
          [ 1.6648e+01,  3.3976e+01,  8.0695e-01,  1.9801e+00,  4.4940e+00,
            1.6297e+00,  7.7214e-04, -2.9091e-04,  1.5930e+00],
          [ 1.6166e+01,  3.1436e+01,  7.3657e-01,  2.0261e+00,  4.7112e+00,
            1.5760e+00,  7.7214e-04, -2.9091e-04,  1.5873e+00],
          [ 1.7128e+01,  3.6593e+01,  1.0299e+00,  2.2086e+00,  5.0573e+00,
            2.0182e+00,  7.7214e-04, -2.9091e-04,  1.5980e+00],
          [ 2.8574e+01,  2.7052e+01,  1.2595e+00,  7.0081e-01,  7.3627e-01,
            1.7227e+00,  1.0292e-01,  1.2176e+00, -2.8538e+00],
     

In [37]:
def draw_lidar_simple(pc, color=None):
    ''' Draw lidar points. simplest set up. '''
    fig = mlab.figure(figure=None, bgcolor=(0,0,0), fgcolor=None, engine=None, size=(1600, 1000))
    if color is None: color = pc[:,2]
    #draw points
    mlab.points3d(pc[:,0], pc[:,1], pc[:,2], color, color=None, mode='point', colormap = 'gnuplot', scale_factor=20, figure=fig)
    #draw origin
    mlab.points3d(0, 0, 0, color=(1,1,1), mode='sphere', scale_factor=0.2)
    #draw axis
    axes=np.array([
        [2.,0.,0.,0.],
        [0.,2.,0.,0.],
        [0.,0.,2.,0.],
    ],dtype=np.float64)
    mlab.plot3d([0, axes[0,0]], [0, axes[0,1]], [0, axes[0,2]], color=(1,0,0), tube_radius=None, figure=fig)
    mlab.plot3d([0, axes[1,0]], [0, axes[1,1]], [0, axes[1,2]], color=(0,1,0), tube_radius=None, figure=fig)
    mlab.plot3d([0, axes[2,0]], [0, axes[2,1]], [0, axes[2,2]], color=(0,0,1), tube_radius=None, figure=fig)
    mlab.view(azimuth=180, elevation=70, focalpoint=[ 12.0909996 , -1.04700089, -2.03249991], distance=62.0, figure=fig)
    return fig

def draw_lidar(pc, color=None, fig=None, bgcolor=(0,0,0), pts_scale=1, pts_mode='point', pts_color=None):
    ''' Draw lidar points
    Args:
        pc: numpy array (n,3) of XYZ
        color: numpy array (n) of intensity or whatever
        fig: mayavi figure handler, if None create new one otherwise will use it
    Returns:
        fig: created or used fig
    '''
    if fig is None: fig = mlab.figure(figure=None, bgcolor=bgcolor, fgcolor=None, engine=None, size=(1600, 1000))
    if color is None: color = pc[:,2]
    mlab.points3d(pc[:,0], pc[:,1], pc[:,2], color, color=pts_color, mode=pts_mode, colormap = 'gnuplot', scale_factor=pts_scale, figure=fig)
    
    #draw origin
    mlab.points3d(0, 0, 0, color=(1,1,1), mode='sphere', scale_factor=0.2)
    
    #draw axis
    axes=np.array([
        [2.,0.,0.,0.],
        [0.,2.,0.,0.],
        [0.,0.,2.,0.],
    ],dtype=np.float64)
    mlab.plot3d([0, axes[0,0]], [0, axes[0,1]], [0, axes[0,2]], color=(1,0,0), tube_radius=None, figure=fig)
    mlab.plot3d([0, axes[1,0]], [0, axes[1,1]], [0, axes[1,2]], color=(0,1,0), tube_radius=None, figure=fig)
    mlab.plot3d([0, axes[2,0]], [0, axes[2,1]], [0, axes[2,2]], color=(0,0,1), tube_radius=None, figure=fig)

    # draw fov (todo: update to real sensor spec.)
    fov=np.array([  # 45 degree
        [20., 20., 0.,0.],
        [20.,-20., 0.,0.],
    ],dtype=np.float64)
    
    mlab.plot3d([0, fov[0,0]], [0, fov[0,1]], [0, fov[0,2]], color=(1,1,1), tube_radius=None, line_width=1, figure=fig)
    mlab.plot3d([0, fov[1,0]], [0, fov[1,1]], [0, fov[1,2]], color=(1,1,1), tube_radius=None, line_width=1, figure=fig)
   
    # draw square region
    TOP_Y_MIN=-20
    TOP_Y_MAX=20
    TOP_X_MIN=0
    TOP_X_MAX=40
    TOP_Z_MIN=-2.0
    TOP_Z_MAX=0.4
    
    x1 = TOP_X_MIN
    x2 = TOP_X_MAX
    y1 = TOP_Y_MIN
    y2 = TOP_Y_MAX
    mlab.plot3d([x1, x1], [y1, y2], [0,0], color=(0.5,0.5,0.5), tube_radius=0.1, line_width=1, figure=fig)
    mlab.plot3d([x2, x2], [y1, y2], [0,0], color=(0.5,0.5,0.5), tube_radius=0.1, line_width=1, figure=fig)
    mlab.plot3d([x1, x2], [y1, y1], [0,0], color=(0.5,0.5,0.5), tube_radius=0.1, line_width=1, figure=fig)
    mlab.plot3d([x1, x2], [y2, y2], [0,0], color=(0.5,0.5,0.5), tube_radius=0.1, line_width=1, figure=fig)
    
    #mlab.orientation_axes()
    mlab.view(azimuth=180, elevation=70, focalpoint=[ 12.0909996 , -1.04700089, -2.03249991], distance=62.0, figure=fig)
    return fig

def draw_gt_boxes3d(gt_boxes3d, fig, color=(1,1,1), labels=None, scores=None, line_width=1, draw_text=True, text_scale=(0.4,0.4,0.4), color_list=None):
    ''' Draw 3D bounding boxes
    Args:
        gt_boxes3d: numpy array (n,8,3) for XYZs of the box corners
        fig: mayavi figure handler
        color: RGB value tuple in range (0,1), box line color
        line_width: box line width
        draw_text: boolean, if true, write box indices beside boxes
        text_scale: three number tuple
        color_list: a list of RGB tuple, if not None, overwrite color.
    Returns:
        fig: updated fig
    ''' 
    num = len(gt_boxes3d)
    for n in range(num):
        if(labels[n] in ['VEHICLE', 'CYCLIST', 'PEDESTRIAN']):
            print(labels[n], end=',')
            b = gt_boxes3d[n]
            if color_list is not None:
                color = color_list[n] 
            if scores is not None: 
                if(scores[n]<0.35):
                    continue
#                 print('printing scores too')
                mlab.text3d(b[4,0], b[4,1], b[4,2], labels[n]+'{:.2f}'.format(scores[n]), scale=text_scale, color=color, figure=fig)
            elif labels is not None: mlab.text3d(b[4,0], b[4,1], b[4,2], labels[n], scale=text_scale, color=color, figure=fig)
            elif draw_text: mlab.text3d(b[4,0], b[4,1], b[4,2], '%d'%n, scale=text_scale, color=color, figure=fig)
            for k in range(0,4):
                #http://docs.enthought.com/mayavi/mayavi/auto/mlab_helper_functions.html
                i,j=k,(k+1)%4
                mlab.plot3d([b[i,0], b[j,0]], [b[i,1], b[j,1]], [b[i,2], b[j,2]], color=color, tube_radius=None, line_width=line_width, figure=fig)

                i,j=k+4,(k+1)%4 + 4
                mlab.plot3d([b[i,0], b[j,0]], [b[i,1], b[j,1]], [b[i,2], b[j,2]], color=color, tube_radius=None, line_width=line_width, figure=fig)

                i,j=k,k+4
                mlab.plot3d([b[i,0], b[j,0]], [b[i,1], b[j,1]], [b[i,2], b[j,2]], color=color, tube_radius=None, line_width=line_width, figure=fig)
    #mlab.show(1)
    #mlab.view(azimuth=180, elevation=70, focalpoint=[ 12.0909996 , -1.04700089, -2.03249991], distance=62.0, figure=fig)
    return fig


In [35]:
def convert_box_to_coordinates(gt_boxes3d):
    new_gt_boxes3d = []
    for det in gt_boxes3d:
        intermediate = []
        ndet = det['box3d_lidar']
        try:
            ndet = ndet.numpy()
        except:
            pass
        for det in ndet:
            box = []
            center_x = det[0]
            center_y = det[1]
            center_z = det[2]
            width = det[3]
            length = det[4]
            height = det[5]
            det[-1] = -det[-1] - np.pi / 2
            rotation_matrix = Quaternion(axis=[0, 0, 1], radians=det[-1]).rotation_matrix
            ## 3D bounding box corners. (Convention: x points forward, y to the left, z up.)
            x_corners = length / 2 * np.array([1,  1,  1,  1, -1, -1, -1, -1])
            y_corners = width / 2 * np.array([1, -1, -1,  1,  1, -1, -1,  1])
            z_corners = height / 2 * np.array([1,  1, -1, -1,  1,  1, -1, -1])
            corners = np.vstack((x_corners, y_corners, z_corners))
            ## Rotate
            corners = np.dot(rotation_matrix, corners)
            corners[0, :] = corners[0, :] + center_x
            corners[1, :] = corners[1, :] + center_y
            corners[2, :] = corners[2, :] + center_z
            intermediate.append(corners.T)
        new_gt_boxes3d.append(np.array(intermediate))
    return(np.array(new_gt_boxes3d))
def convert_detbox_to_coordinates(gt_boxes3d):
    new_gt_boxes3d = []
    for det in gt_boxes3d:
        intermediate = []
        ndet = det['box3d_lidar']
        try:
            ndet = ndet.numpy()
        except:
            pass
        for det in ndet:
            box = []
            center_x = det[0]
            center_y = det[1]
            center_z = det[2]
            width = det[3]
            length = det[4]
            height = det[5]
#             det[-1] = -det[-1] - np.pi / 2
            rotation_matrix = Quaternion(axis=[0, 0, 1], radians=-det[-1] - np.pi / 2).rotation_matrix
            x_corners = length / 2 * np.array([1,  1,  1,  1, -1, -1, -1, -1])
            y_corners = width / 2 * np.array([1, -1, -1,  1,  1, -1, -1,  1])
            z_corners = height / 2 * np.array([1,  1, -1, -1,  1,  1, -1, -1])
            corners = np.vstack((x_corners, y_corners, z_corners))
            ## Rotate
            corners = np.dot(rotation_matrix, corners)
            corners[0, :] = corners[0, :] + center_x
            corners[1, :] = corners[1, :] + center_y
            corners[2, :] = corners[2, :] + center_z
            intermediate.append(corners.T)
        new_gt_boxes3d.append(np.array(intermediate))
    return(np.array(new_gt_boxes3d))